In [5]:
import random
import os
import re
import numpy as np
from pprint import pprint
print(os.getcwd())
from Backend.py_utils import *
from Backend import perf_calc as perf_calc


g:\CompSciProject
g:\CompSciProject


In [10]:
def get_refusal_p3(uncorrected_ref_speed, rwy_slope): #runway slope

    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "refusal-runway-slope.dig"

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
        
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }
    
    runway_slope_scale = []
    result_slope = []

    for slope_scale, xy_pair_slope in data.items():
        xy_pair_slope = data[slope_scale]

        x_values = xy_pair_slope["x"]
        y_values = xy_pair_slope["y"]

        runway_slope_scale.append(slope_scale)
        result_slope.append(np.interp(rwy_slope,x_values, y_values))

    result = np.interp(uncorrected_ref_speed, runway_slope_scale, result_slope)

    return round(result)


In [60]:
get_refusal_p3(118, 0.4)

118

In [164]:
def get_refusal_p4(prev_data, wind_speed, tail_or_head):

    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "wind-velocity-for-refusal.dig"

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
 
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }

    if tail_or_head == "Headwind":
        wind_speed = wind_speed * 0.5 # apply a +50% factor to reported headwind
    elif tail_or_head == "Tailwind":
        wind_speed = wind_speed * 1.5 # apply a +150% factor to reported headwind
    else:
        wind_speed = wind_speed # do nothing if its a crosswind

    runway_windscale = []
    result_wind = []

    for windscale, xy_pair_wind in data.items():
        xy_pair_wind = data[windscale]

        x_values = xy_pair_wind["x"]
        y_values = xy_pair_wind["y"]

        runway_windscale.append(windscale)
        result_wind.append(np.interp(wind_speed, x_values, y_values))
    
    result = np.interp(prev_data, runway_windscale, result_wind)

    return round(result)

In [165]:
get_refusal_p4(118,20, "Headwind")

128

In [208]:
def get_refusal_p9(prev_data, yes_no):  

    TOP_FOLDER = "Backend/chart_dig/completed-takeoff/refusal-and-cef-speed"
    DIG_FILE_NAME = "refusal-anti-skid.dig"

    data = {}

    chart = ParseDig(f'./{TOP_FOLDER}/dig/{DIG_FILE_NAME}')
    for c in  chart.curveNames():
        yVector = [row [1] for row in chart.curve(c)]
        xVector = [row [0] for row in chart.curve(c)]
        scale_number = float(re.sub('[^0-9]','', c.replace("-", "_")))
        
        data[scale_number] = {
            "x": xVector,
            "y": yVector
        }

    if yes_no == True: # if the AS is operational
        yes_no = 0
    elif yes_no == False: # if the AS is not operational
        yes_no = 1
    else: yes_no = 0


    as_scale = []
    result_as = []

    for asscale, xy_pair_as in data.items():
        xy_pair_as = data[asscale]

        x_values = xy_pair_as["x"]
        y_values = xy_pair_as["y"]

        as_scale.append(asscale)
        result_as.append(np.interp(yes_no, x_values, y_values))

    result = np.interp(prev_data,as_scale,result_as)

    return round(result)


In [210]:
get_refusal_p9(115,False)

92